We've looked at the most common unsupervised learning task: dimensionality reduction. In this chapter, we will look at a few more unsupervised learning tasks and algorithms:

- Clustering: the goal is to group similar instances together into clusters (data analysis, customer segmentation, recommender systems, search engines, image segmentation, semi-supervised learning, dimensionality reduction..)
- Anomaly detection: the objective is to learn what “normal” data looks like, and use this to detect abnormal instances (defective items on a production line, new trends in a time series)
- Density estimation: this is the task of estimating the probability density function (PDF) of the random process that generated the dataset (anomaly detection: instances located in very low-density regions are likely to be anomalies. also data analysis, visualization..)

##### 1. Clustering

